In [1]:
pip install backtesting

     |████████████████████████████████| 175 kB 3.3 MB/s eta 0:00:01
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173805 sha256=09d6d2b177d58ab6a3840ef3a4f6584d792ef92ba376181c57f0da21c69b3ab1
  Stored in directory: /Users/derek/Library/Caches/pip/wheels/cd/5c/f7/aafe95c37f8b07f838fb0a8cb3177de23a38c09cbd10b447b8
Successfully built backtesting
Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests
from backtesting import Strategy
from backtesting.lib import crossover
from ratelimit import limits, sleep_and_retry
import config as cfg
import os
import pandas as pd


# Example OHLC daily data for Google Inc.
from backtesting.test import GOOG


In [150]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [113]:
#input: list of symbols
#input: function [ 'TIME_SERIES_WEEKLY']
#input: use ['open','high',low','close','vol']
def save_ts(sym, ts):
    fl = 'ts/'+sym+'.csv' 
    if os.path.exists(fl):
        #add to it
        add = True
    else:
        #create it
        #f = open(fl, "a")
        ts = pd.DataFrame(ts)
        ts.to_csv(fl)
        #f.write(ts.to_string())
        #f.close()
        

cur_pred = pd.read_csv('preds.csv')
cur_pred.columns = cur_pred.columns.str.replace('Unnamed: 0','symbol')
cur_pred['prdd'] = pd.to_datetime(cur_pred['pred_dte'])
scur_pred = cur_pred.sort_values('prdd',ascending=False)
symbolz = scur_pred.head(86)['symbol'].unique()
#symbolz = ['RFACR']

for sym in symbolz:

    dat = get_time_series([sym])
    
    dat1 = pd.DataFrame.from_dict(dat[sym])
    dat1.index = dat1.DATE
    dat1.drop(columns='DATE', inplace=True)
    dat1.columns = ['Open','High','Low','Close','Volume']
    for c in dat1.columns:
        dat1[c] = dat1[c].astype(float)
    save_ts(sym, dat1)

attempt:RFACR
attempt:MINM
attempt:ENTG
attempt:MCACR
attempt:PNACR
attempt:MAXN
attempt:TACT
attempt:LASE
attempt:RMBL
attempt:CELH
attempt:GRWG
attempt:HCDI
attempt:CTIC
attempt:TH
attempt:LMB
attempt:AYTU
attempt:NEO
attempt:PYPS
attempt:CUEN
attempt:NVAX
attempt:MOBQ
attempt:MDGS
attempt:ATLX
attempt:BRTX
attempt:NEOV
attempt:ECX
attempt:ZS
attempt:OKYO
attempt:NSSC
attempt:FUBO
attempt:IMPP
attempt:ML
attempt:GBBKR
attempt:CRBP
attempt:FNKO
attempt:LFVN
attempt:CTLP
attempt:IEP
attempt:RILY
attempt:OMI
attempt:SMFL
attempt:WAL
attempt:SVFD
attempt:BAOS
attempt:SHAK
attempt:NVEC
attempt:PLPC
attempt:WKSP
attempt:ITRI
attempt:GRBK


In [4]:


def get_time_series(symbols, fg = False):
    ts = {}
    key = cfg.alpha_config['key']
    for s in symbols:
        print('attempt:'+s)

        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol='+s+'&apikey='+key
        data = call_api(url)

        if data == []:
            continue
        else:
            if 'Weekly Time Series' in data:
                dat = pd.DataFrame.from_dict(data['Weekly Time Series'], orient='index')
                dat.columns = ['open','high','low','close','vol']
                #dat.head()

                dat['close'] = dat['close'].astype(float)
                dat['DATE'] = pd.to_datetime(dat.index)

                dat_new = dat.set_index('DATE').resample('1D').ffill().reset_index()
                ts[s] = dat_new
                if fg == True:
                    fig = px.line(dat, x='DATE', y='close')
                    fig.show()
            else:
                print('skipped, no weekly series data?')
                print(data)

    return ts

TIME_PERIOD = 12   # time period in seconds

@sleep_and_retry
@limits(calls=1, period=TIME_PERIOD)
def call_api(url):
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception('API response: {}'.format(response.status_code))

    data = response.json()
    if 'Error Message' in data:
        #error with symbol
        #raise Exception('API Symbol error{}'.format(url))
        data = []
    return data


In [123]:
cur_pred.head()

,symbol,2023-05-22 00:00:00,2023-05-27 00:00:00,2023-06-01 00:00:00,2023-06-11 00:00:00,2023-07-11 00:00:00,2023-08-10 00:00:00,pred_dte,2023-05-21 00:00:00,2023-05-26 00:00:00,...,2023-07-13 00:00:00,2023-04-23 00:00:00,2023-06-12 00:00:00,2023-07-12 00:00:00,2023-04-22 00:00:00,2023-04-21 00:00:00,2023-04-20 00:00:00,2023-04-19 00:00:00,2023-04-18 00:00:00,prdd
0,RFACR,0.150141,0.150270,0.150399,0.150658,0.151436,0.152216,2023-05-12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-12
1,MAXN,36.592655,36.710265,36.828387,37.066181,37.792155,38.537515,2023-05-12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-12
2,MINM,6.095319,6.394339,6.702985,7.349759,9.536933,12.120013,2023-05-12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-12
3,ENTG,93.718014,93.862406,94.007057,94.297134,95.173608,96.059537,2023-05-12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-12
4,MCACR,0.131139,0.131712,0.132287,0.133447,0.136997,0.140655,2023-05-12,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-12


In [191]:
import numpy as np

def find_earliest_price(sym, df, exclude_cols=['pred_dte','prdd','symbol']):
    dfm = df[df['symbol']==sym]
    dfm.drop(columns=exclude_cols, inplace=True)
    dfn = dfm.T
    dfn = dfn.dropna(how='all')
    dfn['ddte'] = dfn.index
    dfn.reset_index()
    dfn = dfn.sort_values(['ddte'], ascending=True)
    #print(dfn)
    for j, jrow in dfn.iterrows():
        for c in dfn.columns:
            if not pd.isna(jrow[c]):
                #print(c)
                return jrow[c]
        #print(dfn)

dfq = find_earliest_price('ML', cur_pred)    


<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)


In [201]:
import os
# assign directory
directory = 'ts/'

dates = cur_pred[['symbol','prdd','pred_dte']]
dates = dates.sort_values(by=['prdd'],ascending=True)

cur_pred = cur_pred.sort_values(by=['prdd'],ascending=True)
#iterate our portfolio and their dates
equity = 0
roi = 0
for ri, d in dates.iterrows():
    
    # iterate over files in
    # that directory

    for filename in os.listdir(directory):
        #iterate files
        if filename == d['symbol']+'.csv':
            print('Found symbol file for:'+d['symbol'])
            #if the file is the same symbol in portfolio
            f = os.path.join(directory, filename)
            # checking if it is a file

            if os.path.isfile(f):
                ts = pd.read_csv(f)
                ts['DATE'] = pd.to_datetime(ts['DATE'])
                ts.index = ts['DATE']
                
                ts.drop(columns='DATE', inplace=True)
                ts.columns = ['Open','High','Low','Close','Volume']
                for c in ts.columns:
                    ts[c] = ts[c].astype(float)
                
                ts = ts[ts.index >= d['prdd']]
                #print(ts)
                amt = find_earliest_price(sym, cur_pred, exclude_cols=['pred_dte','prdd','symbol'])
                equity+=amt
                bt = Backtest(ts, SmaCross, cash=amt, commission=.02)
                stats = bt.run()
                roi+=stats[5]
                print('Invested '+str(amt)+' in '+d['symbol']+' on '+d['pred_dte'])
                print('Return %:'+str(stats['Return [%]']))
                print('Return Equity: '+str(stats[5]))


print(equity)
print(roi)

Found symbol file for:GBBKR
Invested 45.85306352639018 in GBBKR on 2023-04-08
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:RMBL
Invested 45.85306352639018 in RMBL on 2023-04-12
Return %:6.859301774220213
Return Equity: 48.99826352639018
Found symbol file for:RFACR
Invested 45.85306352639018 in RFACR on 2023-04-14
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:CRBP
Invested 45.85306352639018 in CRBP on 2023-04-15
Return %:13.569431399973972
Return Equity: 53.625063526390186
Found symbol file for:CRBP


<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
/Users/derek/opt/anaconda3/lib/python3.8/site-packages/backtesting/_stats.py:122: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
<ipython-input-191-da13d628451e>:5: SettingWit

Invested 45.85306352639018 in CRBP on 2023-04-16
Return %:13.569431399973972
Return Equity: 53.625063526390186
Found symbol file for:CRBP
Invested 45.85306352639018 in CRBP on 2023-04-17
Return %:13.569431399973972
Return Equity: 53.625063526390186
Found symbol file for:SVFD
Invested 45.85306352639018 in SVFD on 2023-04-19
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:MDGS
Invested 45.85306352639018 in MDGS on 2023-04-20
Return %:22.396758711856755
Return Equity: 56.122663526390184
Found symbol file for:CTIC
Invested 45.85306352639018 in CTIC on 2023-04-20
Return %:77.5655916196969
Return Equity: 81.41926352639018
Found symbol file for:WAL
Invested 45.85306352639018 in WAL on 2023-04-20
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:WKSP


<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
/Users/derek/opt/anaconda3/lib/python3.8/site-packages/bac

Invested 45.85306352639018 in WKSP on 2023-04-21
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:RFACR
Invested 45.85306352639018 in RFACR on 2023-04-22
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:RFACR
Invested 45.85306352639018 in RFACR on 2023-04-23
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:CRBP
Invested 45.85306352639018 in CRBP on 2023-04-25
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:SMFL
Invested 45.85306352639018 in SMFL on 2023-04-25
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:PNACR
Invested 45.85306352639018 in PNACR on 2023-04-29
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:PNACR
Invested 45.85306352639018 in PNACR on 2023-04-30

...

<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning

Invested 45.85306352639018 in PYPS on 2023-05-10
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:GRWG
Invested 45.85306352639018 in GRWG on 2023-05-11
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:HCDI
Invested 45.85306352639018 in HCDI on 2023-05-11
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:MCACR
Invested 45.85306352639018 in MCACR on 2023-05-11
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:CTIC
Invested 45.85306352639018 in CTIC on 2023-05-11
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:TACT
Invested 45.85306352639018 in TACT on 2023-05-11

<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
<ipython-input-201-49120ab594fc>:39: UserWarning: Some pri


Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:LASE
Invested 45.85306352639018 in LASE on 2023-05-11
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:CELH
Invested 45.85306352639018 in CELH on 2023-05-11
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:RMBL
Invested 45.85306352639018 in RMBL on 2023-05-11
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:PNACR
Invested 45.85306352639018 in PNACR on 2023-05-12
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:MCACR
Invested 45.85306352639018 in MCACR on 2023-05-12
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:ENTG
Invested 45.85306352639018 in ENTG on 2023-05-12
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:MINM
Invested 45.85306352639018 in MINM on 2023-05-12
Return %:0.0
Return Equity: 45.85306352639018
Found symbol file for:MAXN
Invested 45.85306352639018 in MAXN on 2023-05-12
Return %:0.0
Return

<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)
<ipython-input-191-da13d628451e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.drop(columns=exclude_cols, inplace=True)


In [121]:
print(stats)

Start                     2020-09-04 00:00:00
End                       2023-05-12 00:00:00
Duration                    980 days 00:00:00
Exposure Time [%]                   96.330275
Equity Final [$]                220117.830183
Equity Peak [$]                 222186.580183
Return [%]                      440135.660366
Buy & Hold Return [%]                 6.19469
Return (Ann.) [%]                 2168.315554
Volatility (Ann.) [%]     56122647331319.0...
Sharpe Ratio                              0.0
Sortino Ratio                       43.653176
Calmar Ratio                         65.75579
Max. Drawdown [%]                   -32.97528
Avg. Drawdown [%]                   -9.220971
Max. Drawdown Duration       38 days 00:00:00
Avg. Drawdown Duration       11 days 00:00:00
# Trades                                   51
Win Rate [%]                        84.313725
Best Trade [%]                    1496.806387
Worst Trade [%]                     -8.711988
Avg. Trade [%]                    

In [108]:
tt = pd.read_csv('ts/FNKO.csv')

tst.tail()

,0,2,4,5,7,9,13
977,2023-05-08,3.9900,13.9600,,,,None
978,2023-05-09,3.9900,13.9600,,,,None
979,2023-05-10,3.9900,13.9600,,,,None
980,2023-05-11,3.9900,13.9600,,,,None
981,2023-05-12,5.3200,15.8500,,,,None


In [9]:
dat1 = pd.DataFrame.from_dict(dat[sym])
dat1.index = dat1.DATE
dat1.drop(columns='DATE', inplace=True)
dat1.columns = ['Open','High','Low','Close','Volume']
for c in dat1.columns:
    dat1[c] = dat1[c].astype(float)
dat1.tail()

,Open,High,Low,Close,Volume
DATE,,,,,
2023-05-06,51.95,52.450,46.425,47.08,796041.0
2023-05-07,51.95,52.450,46.425,47.08,796041.0
2023-05-08,51.95,52.450,46.425,47.08,796041.0
2023-05-09,51.95,52.450,46.425,47.08,796041.0
2023-05-10,47.26,47.425,42.840,44.52,423817.0


In [10]:
dat2 = dat1.tail(90)

In [11]:
dat1.dtypes

Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
dtype: object

In [12]:
import pandas as pd


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
            
from backtesting import Backtest


bt = Backtest(dat2, SmaCross, cash=50, commission=.002)
stats = bt.run()
stats

<ipython-input-12-c6199097b327>:38: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(dat2, SmaCross, cash=50, commission=.002)


Start                     2023-02-10 00:00:00
End                       2023-05-10 00:00:00
Duration                     89 days 00:00:00
Exposure Time [%]                   68.888889
Equity Final [$]                     60.64322
Equity Peak [$]                      65.33322
Return [%]                           21.28644
Buy & Hold Return [%]               20.650407
Return (Ann.) [%]                  118.728428
Volatility (Ann.) [%]              618.184207
Sharpe Ratio                          0.19206
Sortino Ratio                        2.004552
Calmar Ratio                         3.831952
Max. Drawdown [%]                    -30.9838
Avg. Drawdown [%]                  -14.123747
Max. Drawdown Duration       24 days 00:00:00
Avg. Drawdown Duration       13 days 00:00:00
# Trades                                    3
Win Rate [%]                        66.666667
Best Trade [%]                      45.393553
Worst Trade [%]                    -12.526177
Avg. Trade [%]                    

In [13]:
bt.plot()

Row(id='1434', ...)